<a target="_blank" href="https://colab.research.google.com/github/petuch03/machine-learning-things/blob/master/advanced-ml/hw-1-report.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Report for 1st homework on Advanced ML course @ Neapolis University Pafos
### By Egor Safronov

### EMforDS

The way I completed the task quite direct without any extraordinary things. 
Starting with creation of required indexes and initializing `self.counts`:
```python
usr_to_index = {usr: index for index, usr in enumerate(self.usr_arr)}
obj_to_index = {obj: index for index, obj in enumerate(self.obj_arr)}
lbl_to_index = {lbl: index for index, lbl in enumerate(self.lbl_arr)}

self.counts = np.zeros((self.num_usr, self.num_obj, self.num_lbl))
for _, row in self.crowd_labels.iterrows():
    user_index = usr_to_index[row['user']]
    object_index = obj_to_index[row['object']]
    label_index = lbl_to_index[row['label']]
    self.counts[user_index, object_index, label_index] += 1
```
Also, `pi` and `rho` required proper initialization:
```python
# TODO: calculate self.pi and self.rho initial values
# init pi and rho with (1 / num_lbl)
self.pi = np.full((self.num_usr, self.num_lbl, self.num_lbl), 1 / self.num_lbl)
self.rho = np.full(self.num_lbl, 1 / self.num_lbl)
```
After it, I implemented E and M step of the algorithms. Implementation is quite straightforward, added code marked with original `TODOs` from the task:

```python
def _e_step(self):
    # interation over objects
    for i_obj in tqdm(range(self.num_obj), postfix='e_step', leave=False):
        # TODO: update self.label_prob
        # likelihood pi*counts for the current object, then take the product
        i_obj_count = self.counts[:, [i_obj], :]
        probs = (self.pi ** i_obj_count).prod(axis=0).prod(axis=1)
        # adjust label probabilities
        self.label_prob[i_obj] = probs * self.rho
        # total probability
        total_prob = self.label_prob[i_obj].sum()  # numpy.float64

        self.label_prob[i_obj] /= total_prob
    return self.label_prob

def _m_step(self, posterior):
    self.rho = posterior.sum(axis=0) / posterior.sum()

    for i_lbl in tqdm(range(self.num_lbl), postfix='m_step', leave=True):
        # TODO: update self.pi
        error = self.label_prob[:, i_lbl] @ self.counts
        total_error = np.sum(error, axis=1, keepdims=True)
        for idx in range(total_error.shape[0]):
            if total_error[idx] == 0:
                self.pi[:, i_lbl, :][idx] = total_error[idx]
            else:
                self.pi[:, i_lbl, :][idx] = error[idx] / total_error[idx]
```

### Aggregation

For implementation of majority voting I used crosstab as it was recommended in the task definition:
```python
# TODO: majority voting
crosstab = pd.crosstab(index=labels['object'], columns=labels['label'])

# highest freq for each object
majority_vote = crosstab.idxmax(axis=1)
```

### TopicModeling

As before, nothing special here. The perplexity implementation:
```python
log_likelihood = 0
for d in range(self.num_docs):
    for w in range(self.num_words):
        p_wd = np.dot(self.Phi[w, :], self.Theta[:, d])
        if self.counts[w, d] > 0:
            log_likelihood += self.counts[w, d] * np.log(p_wd)

perplexity = np.exp(-log_likelihood / self.total_words)
```
I slightly modified existing main function, because there were not enough sample (989 instead of 1000). In addition I enriched main with PLSA call itself and building perplexity chart over iterations. The example of how algorithm works:
```
100%|██████████| 989/989 [00:00<00:00, 1141.98it/s]
Iteration 1, Perplexity: 10768.906824213775
Iteration 2, Perplexity: 10616.40668455625
Iteration 3, Perplexity: 10487.47569407712
Iteration 4, Perplexity: 10352.93221999247
Iteration 5, Perplexity: 10203.646477272394
Iteration 6, Perplexity: 10040.59522358958
Iteration 7, Perplexity: 9872.388951368635
Iteration 8, Perplexity: 9710.43408993396
Iteration 9, Perplexity: 9563.663291295508
Iteration 10, Perplexity: 9436.368212821286
Iteration 11, Perplexity: 9328.786985044177
Iteration 12, Perplexity: 9238.741548370568
Iteration 13, Perplexity: 9163.118996773246
Iteration 14, Perplexity: 9099.079984956297
Iteration 15, Perplexity: 9044.73464884072
Iteration 16, Perplexity: 8998.624927363908
Iteration 17, Perplexity: 8959.25318839123
Iteration 18, Perplexity: 8925.255179170925
Iteration 19, Perplexity: 8895.62025030814
Iteration 20, Perplexity: 8869.567290846675
Iteration 21, Perplexity: 8846.387688449287
Iteration 22, Perplexity: 8825.525878778883
Iteration 23, Perplexity: 8806.577363358963
Iteration 24, Perplexity: 8789.178629966347
Iteration 25, Perplexity: 8773.109470298365
Iteration 26, Perplexity: 8758.34828473269
Iteration 27, Perplexity: 8744.917724498344
Iteration 28, Perplexity: 8732.829379287385
Iteration 29, Perplexity: 8722.03794300073
Iteration 30, Perplexity: 8712.413782348347
Topic #1:
на, работы, опыт, по, знание, не, от, умение, работать, лет, работ, навыки, до, требования, участие, документации, рф, тк, будет, график
Topic #2:
работы, 00, контроль, опыт, обязанности, до, условия, требования, от, график, по, организация, ведение, образование, года, заработная, плата, документации, ответственность, 18
Topic #3:
по, на, условия, для, или, требования, обязанности, образование, за, высшее, при, труда, проведение, 30, рф, от, полный, наличие, оформление, тк
Topic #4:
работа, на, работы, обязанности, тк, рф, условия, оформление, требования, возможность, за, месяц, по, график, товара, соответствии, роста, оплата, карьерного, профессионального
Topic #5:
мы, компании, на, для, по, клиентов, возможность, обучение, продаж, россии, сотрудников, от, работа, из, работать, работы, предлагаем, работу, за, компания

```
The perplexity chart over 30 iterations:
![PLSA perplexity over iterations](hw-1.png "PLSA perplexity over iterations")